### Connect to Gremlin Server

In [3]:
import os
import sys
from pathlib import Path

container_src_path = Path('/app/src/')
local_src_path = Path(Path.cwd(), 'src/')

# see if this src path exists.
# if it does, we are in a container.
# if not, we are in local.
if not container_src_path.exists():
    src_path = local_src_path
else:
    src_path = container_src_path

src_path_str = str(src_path)
if src_path_str not in sys.path:
    sys.path.insert(0, src_path_str)

from dotenv import load_dotenv

from graph_populate import g

load_dotenv()

from concurrent.futures import ThreadPoolExecutor
from contextlib import contextmanager
from queue import Queue

# Queue to hold statements
statement_queue = Queue()

# Function to add statement to queue
def add_to_queue(statement):
    statement_queue.put(statement)

@contextmanager
def execute_gremlin_statements():
    try:
        yield add_to_queue
    finally:
        statements = []
        while not statement_queue.empty():
            statements.append(statement_queue.get())
        
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(statement) for statement in statements]
            results = [future.result() for future in futures]

        for result in results:
            print(result)

### Clear Database

In [4]:
with execute_gremlin_statements() as add_statement:

    add_statement(lambda: g.V().drop().iterate())

[['V'], ['drop'], ['none']]


### Execute Some Queries

In [5]:
from concurrent.futures import ThreadPoolExecutor

# def add_vertex():
#     return g.addV('Address').property('addr', 'some_address').next()

# with ThreadPoolExecutor() as executor:
#     future = executor.submit(add_vertex)
#     result = future.result()
    # print(result)
    
with execute_gremlin_statements() as add_statement:
    add_statement(lambda: g.addV('Address').property('addr', 'some_address').next())

v[4288]
